## **Collections - Jupyter**

In [1]:
YEAR = '2022'
MONTH = 'junho'
CAMPAIGN = 'spike'
CAMPAIGN_ENTRY_DATE = '2022-06-12'
CAMPAIGN_EXIT_DATE = '2022-06-12'
CLUSTER_DELIVERY_FOLDER_ID = '1aHhaVTzJAOKB8bL6Urp4z4J8x0k535Gi'

### **Imports**

In [2]:
import os
import time
import re
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
import gspread
import pandas as pd
import numpy as np
from functools import reduce
from gspread_dataframe import set_with_dataframe
from unidecode import unidecode

### **Authentication with Google Drive**

In [3]:
gauth = GoogleAuth()
drive = GoogleDrive(gauth)

gauth.LoadCredentialsFile('../credentials/mycreds.txt')
if gauth.credentials is None:
    gauth.LocalWebserverAuth()
elif gauth.access_token_expired:
    os.remove('../credentials/mycreds.txt')
    gauth.LocalWebserverAuth()
else:
    gauth.Authorize()

gauth.SaveCredentialsFile('../credentials/mycreds.txt')

c:\Users\Sea Group\anaconda3\anaconda\lib\site-packages\oauth2client\_helpers.py:255: UserWarning: Cannot access ../credentials/mycreds.txt: No such file or directory
  warnings.warn(_MISSING_FILE_MESSAGE.format(filename))


### **Functions**

In [4]:
#########################
### gspread functions ###
#########################

# TODO: insert your own credentials file with help of gspread documentation. (https://docs.gspread.org/en/latest/oauth2.html)

def gspread_start(service_account='../credentials/client_secrets.json'):
    """Start gspread authentication.

    Args:
        service_account (str, optional): Path of the Google Cloud service_account json.
        Defaults to '../credentials/gsheets.json'.

    Returns:
        gc: gc is a connection object with the Google Sheets API.
    """
    scope = ['https://spreadsheets.google.com/feeds']
    #credentials = Credentials.from_service_account_file(service_account, scopes=scope)
    gc = gspread.oauth(credentials_filename = "../credentials/credentials.json")
    return gc


def get_spreadsheet_dataframe(spreadsheet_key, worksheet):
    """Stablish the gspread connection and return the dataframe of the spreadsheet.

    Args:
        spreadsheet_key (string): The ID of the Google Spreadsheet
        worksheet (string): The title of the worksheet

    Returns:
        Dataframe: Returns a dataframe of the spreadsheet.
    """
    book = gspread_start().open_by_key(spreadsheet_key)
    sheet = book.worksheet(worksheet)
    table = sheet.get_all_values()
    return pd.DataFrame(table[1:], columns = table[0])


#########################
##### convert list ######
#########################

# We use this function to convert the category column in Parameters file.
def ConvertList(strConv):
    """Convert a string list (example: "apple, fruit, grape") into an actual Python list.

    Args:
        strConv (string): String you want to convert.

    Returns:
        list: Returns a list with all the elements separated by comma.
    """
    list_in = list(strConv.split(","))
    return list_in


#########################
###### sort by cat ######
#########################

def sort_by_cat(df,cat):
    """Algorithm that sorts the elements of a collection according to their categories.

    Args:
        df (Dataframe): The dataframe you want to sort, in this case, the collection.
        cat (Dataframe.column): The column of the dataframe to sort by, in this case, the category.

    Returns:
        Dataframe: Returns the dataframe sorted by category.
    """
    n = df.shape[0]
    df_new = pd.DataFrame()
    for x in range(n):
        for i in df[cat].unique():
            df_apoio = df.loc[df[cat]==i]
            if x <= (df_apoio.shape[0]-1):
                df_apoio = df_apoio.sort_values('score',ascending = False).reset_index()
                df_apoio = df_apoio.iloc[[x]]                
                df_new = pd.concat([df_new, df_apoio])
            else: pass
    return df_new

#########################
#### score functions ####
#########################


# TODO: change score categories
def classifier(df):
    """Algorithm to classify and define score for every element of a collection.

    Args:
        df (Dataframe): The collection dataframe.

    Returns:
        Dataframe: Returns the dataframe, with the score and sorted by the score.
    """
    n = df.shape[0]
    lista = []
    for value in range(n):
        lista.append(scorer.score(df.iloc[value]))
    df['score'] = lista
    df = df.sort_values('score',ascending = False)
    return df


class BaseScorer(object):
    """Class for handling the process of scoring.

    Args:
        object (_type_): _description_
    """
    def __init__(self):
        self._next_scorer = None

    def set_next(self, scorer):
        self._next_scorer = scorer

        return scorer

    def _call_next(self, collection, score):

        if self._next_scorer is None:
            return score

        return self._next_scorer.score(collection, score)

    def score(self, collection, score=0):
        raise NotImplementedError

    @staticmethod
    def chain(scorers):
        reduce(lambda x, y: x.set_next(y), scorers)

        return scorers[0]


class sales(BaseScorer):
    """Class for handling the sales score."

    Args:
        BaseScorer (_type_): _description_
    """

    def score(self, collection, score=0):
        for category in dfs_para.keys():
            n = (np.where(~np.isnan(dfs_para[str(category)]['Score Vendas'])))
            n = n[0].tolist()            
            if (category =='Bags') & (collection.main_category in (['Bags', 'Jewelry & Watches', 'Fashion Accessories'])):
                for i in n:
                    if i is not n[-1]:
                        if (collection.gross_orders >= dfs_para[str(category)]['Valor Vendas'][i]) & (collection.gross_orders < dfs_para[str(category)]['Valor Vendas'][i+1]):
                            score = score + dfs_para[str(category)]['Score Vendas'][i]*dfs_para[str(category)]['Peso Venda'][0]

                    elif (collection.gross_orders >= dfs_para[str(category)]['Valor Vendas'][i]):
                            score = score + dfs_para[str(category)]['Score Vendas'][i]*dfs_para[str(category)]['Peso Venda'][0] 

                return self._call_next(collection, score)

            if (category =='Clothes') & (collection.main_category in (['Women Clothes', 'Men Clothes'])) & (collection.sub_category not in (['Underwear & Sleepwear', 'Socks'])):
                for i in n:
                    if i is not n[-1]:
                        if (collection.gross_orders >= dfs_para[str(category)]['Valor Vendas'][i]) & (collection.gross_orders < dfs_para[str(category)]['Valor Vendas'][i+1]):
                            score = score + dfs_para[str(category)]['Score Vendas'][i]*dfs_para[str(category)]['Peso Venda'][0]

                    elif (collection.gross_orders >= dfs_para[str(category)]['Valor Vendas'][i]):
                            score = score + dfs_para[str(category)]['Score Vendas'][i]*dfs_para[str(category)]['Peso Venda'][0] 

                return self._call_next(collection, score)

            if (category =='Underwear') & (collection.sub_category in (['Underwear & Sleepwear', 'Socks'])):
                for i in n:
                    if i is not n[-1]:
                        if (collection.gross_orders >= dfs_para[str(category)]['Valor Vendas'][i]) & (collection.gross_orders < dfs_para[str(category)]['Valor Vendas'][i+1]):
                            score = score + dfs_para[str(category)]['Score Vendas'][i]*dfs_para[str(category)]['Peso Venda'][0]

                    elif (collection.gross_orders >= dfs_para[str(category)]['Valor Vendas'][i]):
                            score = score + dfs_para[str(category)]['Score Vendas'][i]*dfs_para[str(category)]['Peso Venda'][0] 

                return self._call_next(collection, score)

            if (category =='Shoes') & (collection.main_category == 'Shoes'):
                for i in n:
                    if i is not n[-1]:
                        if (collection.gross_orders >= dfs_para[str(category)]['Valor Vendas'][i]) & (collection.gross_orders < dfs_para[str(category)]['Valor Vendas'][i+1]):
                            score = score + dfs_para[str(category)]['Score Vendas'][i]*dfs_para[str(category)]['Peso Venda'][0]

                    elif (collection.gross_orders >= dfs_para[str(category)]['Valor Vendas'][i]):
                            score = score + dfs_para[str(category)]['Score Vendas'][i]*dfs_para[str(category)]['Peso Venda'][0] 

                return self._call_next(collection, score)
            
            
class rating(BaseScorer):
    """Class for handling the rating score.

    Args:
        BaseScorer (_type_): _description_
    """

    def score(self, collection, score=0):
        for category in dfs_para.keys():
            n = (np.where(~np.isnan(dfs_para[str(category)]['Score Rating'])))
            n = n[0].tolist()            
            if (category =='Bags') & (collection.main_category in (['Bags', 'Jewelry & Watches', 'Fashion Accessories'])):
                for i in n:
                    if i is not n[-1]:
                        if (collection.Rxq >= dfs_para[str(category)]['Valor Rating'][i]) & (collection.Rxq < dfs_para[str(category)]['Valor Rating'][i+1]):
                            score = score + dfs_para[str(category)]['Score Rating'][i]*dfs_para[str(category)]['Peso Rating'][0]

                    elif (collection.Rxq >= dfs_para[str(category)]['Valor Rating'][i]):
                            score = score + dfs_para[str(category)]['Score Rating'][i]*dfs_para[str(category)]['Peso Rating'][0] 

                return self._call_next(collection, score)

            if (category =='Clothes') & (collection.main_category in (['Women Clothes', 'Men Clothes'])) & (collection.sub_category not in (['Underwear & Sleepwear', 'Socks'])):
                for i in n:
                    if i is not n[-1]:
                        if (collection.Rxq >= dfs_para[str(category)]['Valor Rating'][i]) & (collection.Rxq < dfs_para[str(category)]['Valor Rating'][i+1]):
                            score = score + dfs_para[str(category)]['Score Rating'][i]*dfs_para[str(category)]['Peso Rating'][0]

                    elif (collection.Rxq >= dfs_para[str(category)]['Valor Rating'][i]):
                            score = score + dfs_para[str(category)]['Score Rating'][i]*dfs_para[str(category)]['Peso Rating'][0] 

                return self._call_next(collection, score)

            if (category =='Underwear') & (collection.sub_category in (['Underwear & Sleepwear', 'Socks'])):
                for i in n:
                    if i is not n[-1]:
                        if (collection.Rxq >= dfs_para[str(category)]['Valor Rating'][i]) & (collection.Rxq < dfs_para[str(category)]['Valor Rating'][i+1]):
                            score = score + dfs_para[str(category)]['Score Rating'][i]*dfs_para[str(category)]['Peso Rating'][0]

                    elif (collection.Rxq >= dfs_para[str(category)]['Valor Rating'][i]):
                            score = score + dfs_para[str(category)]['Score Rating'][i]*dfs_para[str(category)]['Peso Rating'][0]  

                return self._call_next(collection, score)

            if (category =='Shoes') & (collection.main_category == 'Shoes'):
                for i in n:
                    if i is not n[-1]:
                        if (collection.Rxq >= dfs_para[str(category)]['Valor Rating'][i]) & (collection.Rxq < dfs_para[str(category)]['Valor Rating'][i+1]):
                            score = score + dfs_para[str(category)]['Score Rating'][i]*dfs_para[str(category)]['Peso Rating'][0]

                    elif (collection.Rxq >= dfs_para[str(category)]['Valor Rating'][i]):
                            score = score + dfs_para[str(category)]['Score Rating'][i]*dfs_para[str(category)]['Peso Rating'][0]  

                return self._call_next(collection, score)


class liked_cnt(BaseScorer):
    """Class for handling the liked count score.

    Args:
        BaseScorer (_type_): _description_
    """

    def score(self, collection, score=0):
        for category in dfs_para.keys():
            n = (np.where(~np.isnan(dfs_para[str(category)]['Score Like'])))
            n = n[0].tolist()            
            if (category =='Bags') & (collection.main_category in (['Bags', 'Jewelry & Watches', 'Fashion Accessories'])):
                for i in n:
                    if i is not n[-1]:
                        if (collection.liked_cnt >= dfs_para[str(category)]['Valor Like'][i]) & (collection.liked_cnt < dfs_para[str(category)]['Valor Like'][i+1]):
                            score = score + dfs_para[str(category)]['Score Like'][i]*dfs_para[str(category)]['Peso Like'][0]

                    elif (collection.liked_cnt >= dfs_para[str(category)]['Valor Like'][i]):
                            score = score + dfs_para[str(category)]['Score Like'][i]*dfs_para[str(category)]['Peso Like'][0] 

                return self._call_next(collection, score)

            if (category =='Clothes') & (collection.main_category in (['Women Clothes', 'Men Clothes'])) & (collection.sub_category not in (['Underwear & Sleepwear', 'Socks'])):
                for i in n:
                    if i is not n[-1]:
                        if (collection.liked_cnt >= dfs_para[str(category)]['Valor Like'][i]) & (collection.liked_cnt < dfs_para[str(category)]['Valor Like'][i+1]):
                            score = score + dfs_para[str(category)]['Score Like'][i]*dfs_para[str(category)]['Peso Like'][0]

                    elif (collection.liked_cnt >= dfs_para[str(category)]['Valor Like'][i]):
                            score = score + dfs_para[str(category)]['Score Like'][i]*dfs_para[str(category)]['Peso Like'][0] 

                return self._call_next(collection, score)

            if (category =='Underwear') & (collection.sub_category in (['Underwear & Sleepwear', 'Socks'])):
                for i in n:
                    if i is not n[-1]:
                        if (collection.liked_cnt >= dfs_para[str(category)]['Valor Like'][i]) & (collection.liked_cnt < dfs_para[str(category)]['Valor Like'][i+1]):
                            score = score + dfs_para[str(category)]['Score Like'][i]*dfs_para[str(category)]['Peso Like'][0]

                    elif (collection.liked_cnt >= dfs_para[str(category)]['Valor Like'][i]):
                            score = score + dfs_para[str(category)]['Score Like'][i]*dfs_para[str(category)]['Peso Like'][0]

                return self._call_next(collection, score)

            if (category =='Shoes') & (collection.main_category == 'Shoes'):
                for i in n:
                    if i is not n[-1]:
                        if (collection.liked_cnt >= dfs_para[str(category)]['Valor Like'][i]) & (collection.liked_cnt < dfs_para[str(category)]['Valor Like'][i+1]):
                            score = score + dfs_para[str(category)]['Score Like'][i]*dfs_para[str(category)]['Peso Like'][0]

                    elif (collection.liked_cnt >= dfs_para[str(category)]['Valor Like'][i]):
                            score = score + dfs_para[str(category)]['Score Like'][i]*dfs_para[str(category)]['Peso Like'][0]

                return self._call_next(collection, score)


class item_stock(BaseScorer):
    """Class for handling the item stock score.

    Args:
        BaseScorer (_type_): _description_
    """

    def score(self, collection, score=0):
        for category in dfs_para.keys():
            n = (np.where(~np.isnan(dfs_para[str(category)]['Score Estoque'])))
            n = n[0].tolist()            
            if (category =='Bags') & (collection.main_category in (['Bags', 'Jewelry & Watches', 'Fashion Accessories'])):
                for i in n:
                    if i is not n[-1]:
                        if (collection.item_stock >= dfs_para[str(category)]['Valor Estoque'][i]) & (collection.item_stock < dfs_para[str(category)]['Valor Estoque'][i+1]):
                            score = score + dfs_para[str(category)]['Score Estoque'][i]*dfs_para[str(category)]['Peso Estoque'][0]

                    elif (collection.item_stock >= dfs_para[str(category)]['Valor Estoque'][i]):
                            score = score + dfs_para[str(category)]['Score Estoque'][i]*dfs_para[str(category)]['Peso Estoque'][0] 

                return self._call_next(collection, score)

            if (category =='Clothes') & (collection.main_category in (['Women Clothes', 'Men Clothes'])) & (collection.sub_category not in (['Underwear & Sleepwear', 'Socks'])):
                for i in n:
                    if i is not n[-1]:
                        if (collection.item_stock >= dfs_para[str(category)]['Valor Estoque'][i]) & (collection.item_stock < dfs_para[str(category)]['Valor Estoque'][i+1]):
                            score = score + dfs_para[str(category)]['Score Estoque'][i]*dfs_para[str(category)]['Peso Estoque'][0]

                    elif (collection.item_stock >= dfs_para[str(category)]['Valor Estoque'][i]):
                            score = score + dfs_para[str(category)]['Score Estoque'][i]*dfs_para[str(category)]['Peso Estoque'][0] 

                return self._call_next(collection, score)

            if (category =='Underwear') & (collection.sub_category in (['Underwear & Sleepwear', 'Socks'])):
                for i in n:
                    if i is not n[-1]:
                        if (collection.item_stock >= dfs_para[str(category)]['Valor Estoque'][i]) & (collection.item_stock < dfs_para[str(category)]['Valor Estoque'][i+1]):
                            score = score + dfs_para[str(category)]['Score Estoque'][i]*dfs_para[str(category)]['Peso Estoque'][0]

                    elif (collection.item_stock >= dfs_para[str(category)]['Valor Estoque'][i]):
                            score = score + dfs_para[str(category)]['Score Estoque'][i]*dfs_para[str(category)]['Peso Estoque'][0] 

                return self._call_next(collection, score)

            if (category =='Shoes') & (collection.main_category == 'Shoes'):
                for i in n:
                    if i is not n[-1]:
                        if (collection.item_stock >= dfs_para[str(category)]['Valor Estoque'][i]) & (collection.item_stock < dfs_para[str(category)]['Valor Estoque'][i+1]):
                            score = score + dfs_para[str(category)]['Score Estoque'][i]*dfs_para[str(category)]['Peso Estoque'][0]

                    elif (collection.item_stock >= dfs_para[str(category)]['Valor Estoque'][i]):
                            score = score + dfs_para[str(category)]['Score Estoque'][i]*dfs_para[str(category)]['Peso Estoque'][0] 

                return self._call_next(collection, score)


class promo_price(BaseScorer):
    """Class for handling the promo price score.

    Args:
        BaseScorer (_type_): _description_
    """

    def score(self, collection, score=0):
        for category in dfs_para.keys():
            n = (np.where(~np.isnan(dfs_para[str(category)]['Score Faixa de Preço'])))
            n = n[0].tolist()
            if (category =='Bags') & (collection.main_category in (['Bags', 'Jewelry & Watches', 'Fashion Accessories'])):
                for i in n:
                    if i is n[0]:
                        if (collection.promo_price >= dfs_para[str(category)]['Valor Faixa de Preço'][i]):
                            score = score + 10

                    if i is n[-1]:
                        if (collection.promo_price <= dfs_para[str(category)]['Valor Faixa de Preço'][i]):
                            score = score + dfs_para[str(category)]['Score Faixa de Preço'][i]*dfs_para[str(category)]['Peso Faixa'][0]

                    if i is not (n[0] or n[-1]):
                        if (collection.promo_price <= dfs_para[str(category)]['Valor Faixa de Preço'][i]) & (collection.promo_price > dfs_para[str(category)]['Valor Faixa de Preço'][i+1]):
                            score = score + dfs_para[str(category)]['Score Faixa de Preço'][i]*dfs_para[str(category)]['Peso Faixa'][0] 

                return self._call_next(collection, score)

            if (category =='Clothes') & (collection.main_category in (['Women Clothes', 'Men Clothes'])) & (collection.sub_category not in (['Underwear & Sleepwear', 'Socks'])):
                for i in n:
                    if i is n[0]:
                        if (collection.promo_price >= dfs_para[str(category)]['Valor Faixa de Preço'][i]):
                            score = score + dfs_para[str(category)]['Score Faixa de Preço'][i]*dfs_para[str(category)]['Peso Faixa'][0]

                    if i is n[-1]:
                        if (collection.promo_price <= dfs_para[str(category)]['Valor Faixa de Preço'][i]):
                            score = score + dfs_para[str(category)]['Score Faixa de Preço'][i]*dfs_para[str(category)]['Peso Faixa'][0]

                    if i is not (n[0] or n[-1]):
                        if (collection.promo_price <= dfs_para[str(category)]['Valor Faixa de Preço'][i]) & (collection.promo_price > dfs_para[str(category)]['Valor Faixa de Preço'][i+1]):
                            score = score + dfs_para[str(category)]['Score Faixa de Preço'][i]*dfs_para[str(category)]['Peso Faixa'][0] 


                return self._call_next(collection, score)

            if (category =='Underwear') & (collection.sub_category in (['Underwear & Sleepwear', 'Socks'])):
                for i in n:
                    if i is n[0]:
                        if (collection.promo_price >= dfs_para[str(category)]['Valor Faixa de Preço'][i]):
                            score = score + dfs_para[str(category)]['Score Faixa de Preço'][i]*dfs_para[str(category)]['Peso Faixa'][0]

                    if i is n[-1]:
                        if (collection.promo_price <= dfs_para[str(category)]['Valor Faixa de Preço'][i]):
                            score = score + dfs_para[str(category)]['Score Faixa de Preço'][i]*dfs_para[str(category)]['Peso Faixa'][0]

                    if i is not( n[0] or n[-1]):
                        if (collection.promo_price <= dfs_para[str(category)]['Valor Faixa de Preço'][i]) & (collection.promo_price > dfs_para[str(category)]['Valor Faixa de Preço'][i+1]):
                            score = score + dfs_para[str(category)]['Score Faixa de Preço'][i]*dfs_para[str(category)]['Peso Faixa'][0] 

                return self._call_next(collection, score)

            if (category =='Shoes') & (collection.main_category == 'Shoes'):
                for i in n:
                    if i is n[0]:
                        if (collection.promo_price >= dfs_para[str(category)]['Valor Faixa de Preço'][i]):
                            score = score + dfs_para[str(category)]['Score Faixa de Preço'][i]*dfs_para[str(category)]['Peso Faixa'][0]

                    if i is n[-1]:
                        if (collection.promo_price <= dfs_para[str(category)]['Valor Faixa de Preço'][i]):
                            score = score + dfs_para[str(category)]['Score Faixa de Preço'][i]*dfs_para[str(category)]['Peso Faixa'][0]

                    if i is not (n[0] or n[-1]):
                        if (collection.promo_price <= dfs_para[str(category)]['Valor Faixa de Preço'][i]) & (collection.promo_price > dfs_para[str(category)]['Valor Faixa de Preço'][i+1]):
                            score = score + dfs_para[str(category)]['Score Faixa de Preço'][i]*dfs_para[str(category)]['Peso Faixa'][0] 
                return self._call_next(collection, score)


class brl_gmv(BaseScorer):
    """Class for handling the GMV score.

    Args:
        BaseScorer (_type_): _description_
    """
    def score(self, collection, score=0):
        for category in dfs_para.keys():
            n = (np.where(~np.isnan(dfs_para[str(category)]['Score GMV'])))
            n = n[0].tolist()            
            if (category =='Bags') & (collection.main_category in (['Bags', 'Jewelry & Watches', 'Fashion Accessories'])):
                for i in n:
                    if i is not n[-1]:
                        if (collection.brl_gmv >= dfs_para[str(category)]['Valor GMV'][i]) & (collection.brl_gmv < dfs_para[str(category)]['Valor GMV'][i+1]):
                            score = score + dfs_para[str(category)]['Score GMV'][i]*dfs_para[str(category)]['Peso GMV'][0]

                    elif (collection.brl_gmv >= dfs_para[str(category)]['Valor GMV'][i]):
                            score = score + dfs_para[str(category)]['Score GMV'][i]*dfs_para[str(category)]['Peso GMV'][0] 

                return self._call_next(collection, score)

            if (category =='Clothes') & (collection.main_category in (['Women Clothes', 'Men Clothes'])) & (collection.sub_category not in (['Underwear & Sleepwear', 'Socks'])):
                for i in n:
                    if i is not n[-1]:
                        if (collection.brl_gmv >= dfs_para[str(category)]['Valor GMV'][i]) & (collection.brl_gmv < dfs_para[str(category)]['Valor GMV'][i+1]):
                            score = score + dfs_para[str(category)]['Score GMV'][i]*dfs_para[str(category)]['Peso GMV'][0]

                    elif (collection.brl_gmv >= dfs_para[str(category)]['Valor GMV'][i]):
                            score = score + dfs_para[str(category)]['Score GMV'][i]*dfs_para[str(category)]['Peso GMV'][0]

                return self._call_next(collection, score)

            if (category =='Underwear') & (collection.sub_category in (['Underwear & Sleepwear', 'Socks'])):
                for i in n:
                    if i is not n[-1]:
                        if (collection.brl_gmv >= dfs_para[str(category)]['Valor GMV'][i]) & (collection.brl_gmv < dfs_para[str(category)]['Valor GMV'][i+1]):
                            score = score + dfs_para[str(category)]['Score GMV'][i]*dfs_para[str(category)]['Peso GMV'][0]

                    elif (collection.brl_gmv >= dfs_para[str(category)]['Valor GMV'][i]):
                            score = score + dfs_para[str(category)]['Score GMV'][i]*dfs_para[str(category)]['Peso GMV'][0]

                return self._call_next(collection, score)

            if (category =='Shoes') & (collection.main_category == 'Shoes'):
                for i in n:
                    if i is not n[-1]:
                        if (collection.brl_gmv >= dfs_para[str(category)]['Valor GMV'][i]) & (collection.brl_gmv < dfs_para[str(category)]['Valor GMV'][i+1]):
                            score = score + dfs_para[str(category)]['Score GMV'][i]*dfs_para[str(category)]['Peso GMV'][0]

                    elif (collection.brl_gmv >= dfs_para[str(category)]['Valor GMV'][i]):
                            score = score + dfs_para[str(category)]['Score GMV'][i]*dfs_para[str(category)]['Peso GMV'][0]

                return self._call_next(collection, score)


class base(BaseScorer):
    
    """Class for handling the type of the deal score.

    Args:
        BaseScorer (_type_): _description_
    """

    def score(self, collection, score=0):
        for category in dfs_para.keys():
            n = (np.where(~np.isnan(dfs_para[str(category)]['Score Base'])))
            n = n[0].tolist()            
            if (category =='Bags') & (collection.main_category in (['Bags', 'Jewelry & Watches', 'Fashion Accessories'])):
                for i in n:
                    if (collection.base == dfs_para[str(category)]['Valor Base'][i]):
                        score = score + dfs_para[str(category)]['Score Base'][i]*dfs_para[str(category)]['Peso Base'][0] 

                return self._call_next(collection, score)

            if (category =='Clothes') & (collection.main_category in (['Women Clothes', 'Men Clothes'])) & (collection.sub_category not in (['Underwear & Sleepwear', 'Socks'])):
                for i in n:
                    if (collection.base == dfs_para[str(category)]['Valor Base'][i]):
                        score = score + dfs_para[str(category)]['Score Base'][i]*dfs_para[str(category)]['Peso Base'][0] 

                return self._call_next(collection, score)

            if (category =='Underwear') & (collection.sub_category in (['Underwear & Sleepwear', 'Socks'])):
                for i in n:
                    if (collection.base == dfs_para[str(category)]['Valor Base'][i]):
                        score = score + dfs_para[str(category)]['Score Base'][i]*dfs_para[str(category)]['Peso Base'][0] 

                return self._call_next(collection, score)

            if (category =='Shoes') & (collection.main_category == 'Shoes'):
                for i in n:
                    if (collection.base == dfs_para[str(category)]['Valor Base'][i]):
                        score = score + dfs_para[str(category)]['Score Base'][i]*dfs_para[str(category)]['Peso Base'][0] 

                return self._call_next(collection, score)

scorer = BaseScorer.chain([
    sales(),
    rating(),
    liked_cnt(),
    item_stock(),
    promo_price(),
    brl_gmv(),
    base()
])

# score file
dfs_para = pd.read_excel('../data/score_collections.xlsx', sheet_name=None)


### **Parameters file: loading and treating**

In [5]:
# loading parameters file
df_para = pd.read_excel('../campaigns/'+YEAR+'/'+MONTH+'/'+CAMPAIGN+'/parameters-'+CAMPAIGN+'.xlsx')
df_para.set_index(['collection'], inplace=True)


# treating int columns
int_col = ['min_stock', 'max_items']
df_para[int_col] = df_para[int_col].apply(lambda x: x.astype('int64'))

# treating float columns
float_col = ['min_discount', 'max_discount', 'min_price', 'max_price']
df_para[float_col] = df_para[float_col].apply(lambda x: x.astype('float'))

### **Stock file: loading and treating**

In [6]:
# loading stock
estoque_or = pd.read_csv('../campaigns/'+YEAR+'/'+MONTH+'/'+CAMPAIGN+'/stock-'+CAMPAIGN+'.csv')

# TODO: change cluster category
estoque_or = estoque_or[estoque_or["cluster_category"] == "Fashion"]

# rename new category tree to use the old ones in order to have consistent items
estoque_or.rename(columns={'main_category': 'new_main_category',
                           'sub_category': 'new_sub_category',
                           'level3_category': 'new_level3_category',
                           'old_category': 'main_category',
                           'old_sub_category': 'sub_category',
                           'level3_old_category':'level3_category',
                           'item_id': 'itemid_e',
                           'price': 'original_price',
                           'stock': 'item_stock'},
                  inplace=True)

estoque_or = estoque_or[['itemid_e', 'main_category', 'sub_category', 'level3_category', 'original_price', 'item_stock',
                         'rating_total_cnt', 'liked_cnt', 'rating_score', 'item_name', 'shop_id']]

estoque_or['rating_score'] = estoque_or['rating_score'].fillna(0)


### **Sales file: loading and treating**

In [7]:
# loading
venda_or = pd.read_csv('../campaigns/'+YEAR+'/'+MONTH+'/'+CAMPAIGN+'/sales-'+CAMPAIGN+'.csv')

# treating
venda_or.rename(columns={'item_id': 'itemid_v'}, inplace=True)
venda_or = venda_or[['itemid_v', 'units_sold', 'brl_gmv', 'gross_orders']]

### **Deal pool: loading and treating**

In [8]:
# Loading deal pool
# TODO: change spreadsheet_key and worksheet according to your deal pool

sku_negociados_original = get_spreadsheet_dataframe(
    '1mASRUOvyjJ0dtU43xhnKj4iEN02uWypxIQsXWAHSXRA',
    'Deal Pool')

# Treating deal pool
sku_negociados_or = sku_negociados_original.copy(deep=False)
print('Total deals in deal pool:', sku_negociados_or.shape[0])

# Treating is_collection and campaign dates
CAMPAIGN_START_DATE_CONDITION = sku_negociados_or['start_date'] == CAMPAIGN_ENTRY_DATE
CAMPAIGN_END_DATE_CONDITION = sku_negociados_or['end_date'] == CAMPAIGN_EXIT_DATE
IS_COLLECTION_CONDITION = sku_negociados_or['collections'] == 'Sim'
sku_negociados_or = sku_negociados_or[CAMPAIGN_START_DATE_CONDITION & CAMPAIGN_END_DATE_CONDITION & IS_COLLECTION_CONDITION]
sku_negociados_or.drop_duplicates('item_id', inplace=True)
print('Total deals within campaign dates:', sku_negociados_or.shape[0])

# Treating blank RM and prices
RM_IS_BLANK = sku_negociados_or['rm_name'] == ''
PRICE_IS_BLANK = sku_negociados_or['deal_price_after_rebate'] == {'', '.'}
sku_negociados_or.drop(sku_negociados_or.loc[RM_IS_BLANK].index, inplace=True)
sku_negociados_or.drop(sku_negociados_or.loc[PRICE_IS_BLANK].index, inplace=True)
sku_negociados_or['deal_price_after_rebate'] = pd.to_numeric(sku_negociados_or['deal_price_after_rebate'],
                                                             errors='coerce',
                                                             downcast='float')
sku_negociados_or.dropna(subset=['deal_price_after_rebate'], inplace=True)

# Treating price formatting
deal_price_list = sku_negociados_or['deal_price_after_rebate'].tolist()
deal_price_list = [str(i) for i in deal_price_list]
deal_price_list = [i.replace('R$', '').replace(',', '.').strip() for i in deal_price_list]
deal_price_list = [float(i) for i in deal_price_list]
sku_negociados_or['deal_price_after_rebate'] = deal_price_list

# Treating item_id
sku_negociados_or['item_id'] = pd.to_numeric(sku_negociados_or['item_id'], errors='coerce')
sku_negociados_or.dropna(subset=['item_id'], inplace=True)
sku_negociados_or['item_id'] = sku_negociados_or['item_id'].astype('float').astype('int64')

# Treating blank stock values
df_item_id_promo_stock = sku_negociados_or[['item_id', 'promo_stock']]
lista = ['']
lista = df_item_id_promo_stock.loc[df_item_id_promo_stock['promo_stock'].isin(lista)]['item_id'].tolist()
BLANK_STOCK = estoque_or['itemid_e'].isin(lista)
estoque_null = estoque_or[BLANK_STOCK]
estoque_null = estoque_null[['itemid_e', 'item_stock']]
estoque_null.rename(columns={'itemid_e': 'itemid_n', 'item_stock': 'item_stock_n'}, inplace=True)
sku_negociados_or = pd.merge(sku_negociados_or, estoque_null, how='left', left_on='item_id', right_on='itemid_n')
sku_negociados_or.loc[sku_negociados_or['promo_stock'] == '', 'promo_stock'] = sku_negociados_or['item_stock_n']
sku_negociados_or = pd.merge(sku_negociados_or, estoque_or, left_on='item_id', right_on='itemid_e', how='left')

# treating missing price values
lista = sku_negociados_or['original_price'].tolist()
lista = [str(i) for i in lista]
sku_negociados_or['original_price'] = lista
sku_negociados_or.drop(sku_negociados_or.loc[sku_negociados_or['original_price'] == 'nan'].index, inplace=True)

# creating discount column
lista = sku_negociados_or['original_price'].tolist()
lista = [float(i) for i in lista]
sku_negociados_or['original_price'] = lista
sku_negociados_or['discount'] = (1 - (sku_negociados_or['deal_price_after_rebate'] / sku_negociados_or['original_price']))

# Merging deal pool with sales df
sku_negociados_or.sort_values('item_id', inplace=True, ignore_index=True)
venda_or.sort_values('itemid_v', inplace=True, ignore_index=True)
sku_negociados_or = pd.merge(sku_negociados_or, venda_or, left_on='item_id', right_on='itemid_v', how='left')
sku_negociados_or['units_sold'] = sku_negociados_or['units_sold'].fillna(0)
sku_negociados_or['gross_orders'] = sku_negociados_or['gross_orders'].fillna(0)

# Renaming columns
columns_names_sku = {
    'start_date':'entry_day',
    'end_date':'exit_day',
    'shop_id':'shopid',
    'item_id':'itemid',
    'deal_price_after_rebate':'promo_price',
    'promo_stock':'campaign_stock',
    'collections':'is_collection',
    'rm_name':'RM'
    }

sku_negociados_or.rename(columns=columns_names_sku, inplace=True)

# Selecting columns
sku_negociados_or = sku_negociados_or[
        ['exit_day', 'entry_day', 'shopid',
         'itemid', 'main_category', 'sub_category',
         'level3_category', 'original_price','promo_price',
         'discount', 'campaign_stock', 'item_stock',
         'is_collection', 'item_name','units_sold',
         'rating_total_cnt', 'liked_cnt', 'brl_gmv',
         'gross_orders', 'rating_score', 'RM', 'collection_fs']]

# Treating stock values
sku_negociados_or['new_estoque'] = sku_negociados_or['campaign_stock'].str.extract('(\d+)')
sku_negociados_or.loc[sku_negociados_or['new_estoque'].isnull(), 'new_estoque'] = sku_negociados_or['campaign_stock']
sku_negociados_or['new_estoque'] = pd.to_numeric(sku_negociados_or['new_estoque'], errors='coerce')
sku_negociados_or['campaign_stock'] = sku_negociados_or['new_estoque']
sku_negociados_or.drop(columns='new_estoque', inplace=True)
lista = sku_negociados_or['campaign_stock'].tolist()
lista = [str(i) for i in lista]
sku_negociados_or['campaign_stock'] = lista
sku_negociados_or.drop(sku_negociados_or.loc[sku_negociados_or['campaign_stock'] == 'nan'].index, inplace=True)
sku_negociados_or['campaign_stock'] = sku_negociados_or['campaign_stock'].astype('float').astype('int64')

# Treating dates
sku_negociados_or['entry_day'] = sku_negociados_or['entry_day'].apply(pd.to_datetime, errors='coerce')
sku_negociados_or['exit_day'] = sku_negociados_or['exit_day'].apply(pd.to_datetime, errors='coerce')


# Adding rating column
sku_negociados_or['Rxq'] = sku_negociados_or['rating_total_cnt'] * sku_negociados_or['rating_score']

# Treating shopid
sku_negociados_or['shopid'] = sku_negociados_or['shopid'].astype(np.int64)


print('Total deals after treating:', sku_negociados_or.shape[0])

Total deals in deal pool: 208015
Total deals within campaign dates: 27901
Total deals after treating: 26800


### **Self nominated deals: loading and treating**

In [9]:
# Loading self nominated deals
try:
    self_nominated_original = pd.read_csv('../campaigns/'+YEAR+'/'+MONTH+'/'+CAMPAIGN+'/self-'+CAMPAIGN+'.csv')
except FileNotFoundError:
    self_nominated_original = pd.read_excel('../campaigns/'+YEAR+'/'+MONTH+'/'+CAMPAIGN+'/self-'+CAMPAIGN+'.xlsx')

# TODO: change L1 categories
l1_categories = [
    'Baby & Kids Fashion', 'Bags', 'Fashion Accessories',
    'Men Clothes', 'Shoes', 'Underwear & Sleepwear',
    'Women Clothes'
    ]

self_nominated_or = self_nominated_original.copy(deep=True)
IS_CLUSTER_CATEGORIES = self_nominated_or['main_category_id'].isin(l1_categories)
self_nominated_or = self_nominated_or[IS_CLUSTER_CATEGORIES]

self_nominated_or = self_nominated_or[['shop_id', 'item_id', 'review_status', 'promotion_price']]
self_nominated_or.rename(columns={'item_id': 'itemid', 'shop_id': 'shopid', 'promotion_price': 'promo_price'}, inplace=True)

# Cross with stock and sales base in order to get categories and other useful info
self_nominated_or = pd.merge(self_nominated_or, estoque_or, left_on='itemid', right_on='itemid_e', how='inner')
self_nominated_or = pd.merge(self_nominated_or, venda_or, left_on='itemid', right_on='itemid_v', how='left')

# Dropping columns
self_nominated_or.drop(['itemid_e', 'itemid_v'], axis=1, inplace=True)

# Promo_price treating
promo_price_list = self_nominated_or['promo_price'].tolist()
promo_price_list = [str(i) for i in promo_price_list]
promo_price_list = [i.replace(',', '.') for i in promo_price_list]
promo_price_list = [float(i) for i in promo_price_list]
self_nominated_or['promo_price'] = promo_price_list

# Discount column
self_nominated_or['discount'] = 1 - (self_nominated_or['promo_price'] / self_nominated_or['original_price'])

# Adding Rxq column
self_nominated_or['Rxq'] = self_nominated_or['rating_total_cnt'] * self_nominated_or['rating_score']

### **BI: loading and treating**

In [10]:
# Loading BI
bi_original = pd.read_csv('../campaigns/'+YEAR+'/'+MONTH+'/'+CAMPAIGN+'/bi-'+CAMPAIGN+'.csv')

bi_or = bi_original.copy(deep=False)
bi_or.rename(columns={
    'item_price_before_discount': 'original_price',
    'item_price': 'promo_price',
    'shop_id': 'shopid',
    'item_id': 'itemid',
    'liked_count': 'liked_cnt'
    },
    inplace=True)

# Treating columns type
bi_or['promo_price'] = bi_or['promo_price'].astype('float')
bi_or['rating_score'] = bi_or['rating_score'].astype('float')
bi_or['original_price'] = bi_or['original_price'].astype('float')
bi_or['item_stock'] = bi_or['item_stock'].astype('int64')
bi_or['liked_cnt'] = bi_or['liked_cnt'].astype('int64')
bi_or["rating_count"] = bi_or["rating_count"].astype('int64')
bi_or['Rxq'] = bi_or['rating_count'] * bi_or['rating_score']

# Creating discount column
bi_or['discount'] = 1 - (bi_or['promo_price'] / bi_or['original_price'])

# Getting sales info
bi_or = pd.merge(bi_or, venda_or, left_on='itemid', right_on='itemid_v', how='inner')

### **Blacklists: loading and treating**

In [11]:
def unidecode_lower_list(list_to_unidecode):
    list_to_unidecode = [x.lower() for x in list_to_unidecode]
    list_to_unidecode = [unidecode(x) for x in list_to_unidecode]
    unidecoded_list = list(set(list_to_unidecode))
    return unidecoded_list

blacklist_spreadsheet_key = '1pwe-z4SVuiV-nlSLi0rj3Pe8i1BS9v3ReEU33OazizI'

# blackword
blackword = unidecode_lower_list(get_spreadsheet_dataframe(blacklist_spreadsheet_key, 'blackword').word.values.tolist())

# new users
new_users_item_list = get_spreadsheet_dataframe(blacklist_spreadsheet_key, 'new_users').item_id.values.tolist()

# penalty sellers
penalty_sellers_shop_list = get_spreadsheet_dataframe(blacklist_spreadsheet_key, 'penalty_sellers').shop_id.values.tolist()

# transportadoras
shop_list_3pl = get_spreadsheet_dataframe(blacklist_spreadsheet_key, '3pl').shop_id.values.tolist()

### **Main looping**

In [12]:
def engine_collections():
    
    microsite_id_list = []
    repeated_item_ban_list = []
    
    sku_negociados = sku_negociados_or.copy()
    self_nominated = self_nominated_or.copy()
    bi = bi_or.copy()
    sku_negociados['base'] = 'SKU'
    self_nominated['base'] = 'SELF'
    bi['base'] = 'BI'
    df_blank_collections = pd.DataFrame()
    df_original = pd.DataFrame()
    
    print('Starting...')
    
    df_original = pd.concat([sku_negociados, self_nominated, bi], ignore_index=True)
    print('Total deals (SKU, self and BI):', df_original.shape[0])
    
    df_original['item_name_unidecode'] = df_original['item_name'].apply(lambda x: unidecode(x))
    df_original = df_original[~df_original.item_name_unidecode.str.contains('|'.join(blackword), case=False, flags=re.IGNORECASE, na=False, regex=True)]    
    df_original.sort_values('base', ascending=False, inplace=True)
    df_original.drop_duplicates(subset=["shopid", "itemid"], keep='first', inplace=True)
    print('Blackword filter:', df_original.shape[0])
    
    # TODO: change L1 categories
    l1 = [
            'Baby & Kids Fashion', 'Bags', 'Fashion Accessories',
            'Men Clothes', 'Shoes', 'Underwear & Sleepwear',
            'Women Clothes', 'Jewelry and Watches', 'Jewelry & Watches'
            ]
    
    df_original = df_original[df_original['main_category'].isin(l1_categories)]
    print('Only cluster category filter:', df_original.shape[0])
    
    df_original = df_original[~df_original.itemid.isin(new_users_item_list)]
    print('New users filter:', df_original.shape[0])
    
    df_original = df_original[~df_original.shopid.isin(penalty_sellers_shop_list)]
    print('Penalty sellers filter', df_original.shape[0])
    
    for index, df_parameters in df_para.iterrows():
        # Code that avoid repeated items across the same microsite
        if df_parameters['microsite_id'] in microsite_id_list:
            pass
        else:
            microsite_id_list.clear()
            repeated_item_ban_list.clear()
            microsite_id_list.append(df_parameters['microsite_id'])
        
        collection_name = str(df_parameters.microsite_name) +' - '+ str(df_parameters.name)
        print('\n######### Generating', collection_name, '#########')
        
        df = df_original.copy()
                
        try:
            
            #########################
            #### repeated_items #####
            #########################
            
            df = df[~df.itemid.isin(repeated_item_ban_list)]
            print('Repeated items across microsite drop:', df.shape[0])

            #########################
            ####### insert_bi #######
            #########################
                        
            if df_parameters['insert_bi'] == True:
                print('Collection with BI:', df.shape[0])
            else:
                df = df[~(df['base'] == 'BI')]
                print('Collection without BI:', df.shape[0])
            
            #########################
            ######### price #########
            #########################
            
            if pd.notna(df_parameters['min_price']):
                DF_ABOVE_MIN_PRICE = df['promo_price'] >= df_parameters['min_price']
                DF_BELOW_MAX_PRICE = df['promo_price'] <= df_parameters['max_price']
                DF_BETWEEN_MIN_AND_MAX_PRICE = (DF_ABOVE_MIN_PRICE & DF_BELOW_MAX_PRICE)
                df = df[DF_BETWEEN_MIN_AND_MAX_PRICE]
                print('Price filter:', df.shape[0])
                            
            #########################
            ####### discount ########
            #########################       

            if pd.notna(df_parameters['min_discount']):
                DF_ABOVE_MIN_DISCOUNT = df['discount'] >= df_parameters['min_discount']
                DF_BELOW_MAX_DISCOUNT = df['discount'] <= df_parameters['max_discount']
                DF_BETWEEN_MIN_AND_MAX_DISCOUNT = (DF_ABOVE_MIN_DISCOUNT & DF_BELOW_MAX_DISCOUNT)
                df = df[DF_BETWEEN_MIN_AND_MAX_DISCOUNT]
                print('Discount filter:', df.shape[0])
                        
            #########################
            ####### category ########
            #########################  
            
            if pd.notna(df_parameters['main_category']):
                category_list = ConvertList(df_parameters['main_category'])
                DF_ISIN_MAIN_CATEGORY = df['main_category'].isin(category_list)
                df = df[DF_ISIN_MAIN_CATEGORY]
            
            if pd.notna(df_parameters['sub_category']):
                sub_category_list = ConvertList(df_parameters['sub_category'])
                DF_ISIN_SUB_CATEGORY = df['sub_category'].isin(sub_category_list)
                df = df[DF_ISIN_SUB_CATEGORY]
            
            if pd.notna(df_parameters['l3_category']):
                l3_category_list = ConvertList(df_parameters['l3_category'])
                DF_ISIN_L3_CATEGORY = df['level3_category'].isin(l3_category_list)
                df = df[DF_ISIN_L3_CATEGORY]
            
            print('Category filter:', df.shape[0])
            
            #########################
            ####### keywords ########
            #########################
            
            if pd.notna(df_parameters['keywords']):
                keywords_list = ConvertList(df_parameters['keywords'])
                keywords_list = [unidecode(x) for x in keywords_list]
                df = df[df.item_name_unidecode.str.contains('|'.join(keywords_list), case=False, flags=re.IGNORECASE, na=False, regex=True)]
                print('Keywords filter:', df.shape[0])
                        
            #########################
            ########## 3pl ##########
            #########################
            
            if df_parameters['3pl'] == True:
                df = df[df.shopid.isin(shop_list_3pl)]
                print('3pl filter:', df.shape[0])
                                
            #########################
            ####### max_items #######
            #########################
            
            if pd.notna(df_parameters['max_items']):
                df = df.iloc[:df_parameters['max_items']]
                print('Max items filter:', df.shape[0])
            else:
                df = df.iloc[:1000]
                print('Max items filter:', df.shape[0])
                 
            #########################
            ######### score #########
            #########################       
            
            print('Applying score...', df.shape[0])
            df = classifier(df)
            
            #########################
            ###### sort by cat ######
            #########################  
            
            print('Sorting by category...', df.shape[0])
            # Sorting by category
            if df.sub_category.unique().shape[0] <= 3:
                df = sort_by_cat(df,'level3_category')
            else:
                df = sort_by_cat(df,'sub_category')
                
            #########################
            ### best_seller_order ###
            #########################
            
            if df_parameters['best_seller_order'] == True:
                print('Sorting by gross_orders...', df.shape[0])
                df.sort_values('gross_orders', ascending=False, inplace=True)
            
            #########################
            ### rank/final columns ##
            #########################
                  
            # Creating rank column
            print('Creating rank column...', df.shape[0])
            df.reset_index(drop = True, inplace = True)
            df['rank'] = df.index + 1
                    
            # Final columns
            df['cluster'] = df_parameters['cluster']
            df['microsite_id'] = df_parameters['microsite_id']
            df['collection_id'] = df_parameters['collection_id']
            
            print('Applying final columns...', df.shape[0])
            final_columns = [
                'shopid',
                'itemid',
                'rank',
                'base',
                'item_name',
                'main_category',
                'cluster',
                'microsite_id',
                'collection_id',
                'score',
                'promo_price',
                'gross_orders',
                'collection_fs',
                'original_price',
                'discount'
            ]
            
            df = df[final_columns]
            
            first100items = df.head(50).itemid.values.tolist()
            repeated_item_ban_list.extend(first100items)
            
        except KeyError as e:
            print('Collection without items. Going to the next.')
            continue
        
        #########################
        ##### drive upload ######
        #########################
        
        # Try to send collection to Google Drive
        collection_already_exists = False
        success = False
        sleep_time = 2
        
        while not success:
            try:
                delivery_folder_file_list = drive.ListFile({'q': f"'{CLUSTER_DELIVERY_FOLDER_ID}' in parents and trashed=false"}).GetList()
                success = True
            except Exception as e:
                print('Connection failed. Exception:', e)
        
        collection_already_exists = False        
        for collection in delivery_folder_file_list:
            sleep_time = 2
            success = False
            while not success:
                try:
                    if (str(str(df_parameters['collection_id']) + " - " + str(df_parameters.name)) == str(collection['title'])):
                        collection_already_exists = True
                        print('Collection already exists. Updating it...')
                        spreadsheet_id = collection['id']
                        gc = gspread_start()
                        sh = gc.open_by_key(spreadsheet_id)
                        worksheet_collection = sh.get_worksheet(0)
                        worksheet_collection.clear()
                        set_with_dataframe(worksheet_collection, df, col=1, row=1)
                        print('Collection updated.')
                    success = True
                except Exception as e:
                    print('Fail. Exception occurred. Waiting ' + str(sleep_time) + ' seconds to try again.')
                    time.sleep(sleep_time)
                    sleep_time *= 2
                
        if not collection_already_exists:
            sleep_time = 2
            success = False
            while not success:
                try:
                    print('Creating collection file...')
                    file = drive.CreateFile({'title': str(str(df_parameters['collection_id']) + " - " + str(df_parameters.name)),
                                 'mimeType': 'application/vnd.google-apps.spreadsheet',
                                 'parents': [{'id': CLUSTER_DELIVERY_FOLDER_ID}]})
                    
                    file.Upload()
                    
                    file.InsertPermission({
                    'type': 'domain',
                    'value': 'shopee.com',
                    'role': 'writer'
                    })
                    
                    file.InsertPermission({
                    'type': 'user',
                    'value': 'sheetsvini@sheetsvini.iam.gserviceaccount.com',
                    'role': 'writer'
                    })
                    
                    spreadsheet_id = file['id']
                    
                    gc = gspread_start()
                    
                    sh = gc.open_by_key(spreadsheet_id)
                    
                    worksheet_collection = sh.get_worksheet(0)
                    
                    set_with_dataframe(worksheet_collection, df, col=1, row=1)
                    success = True
                    print('Collection created.')
                    
                except Exception as e:
                    print('Fail. Exception occurred. Waiting ' + str(sleep_time) + ' seconds to try again.')
                    time.sleep(sleep_time)
                    sleep_time *= 2
    print('########## END of collections. ##########')

In [13]:
engine_collections()

Starting...
Total deals (SKU, self and BI): 445631
Blackword filter: 353510
Only cluster category filter: 205735
New users filter: 205735
Penalty sellers filter 205735

######### Generating 8 12 06 Spike Day Guide - Up to R$10 entre R$5 e R$10 #########
Repeated items across microsite drop: 205735
Collection with BI: 205735
Price filter: 8288
Category filter: 6198
Max items filter: 450
Applying score... 450
Sorting by category... 450
Creating rank column... 450
Applying final columns... 450
Creating collection file...
Collection created.

######### Generating 9 12 06 Spike Day Guide - Up to R$30 #########
Repeated items across microsite drop: 205735
Collection with BI: 205735
Price filter: 58912
Category filter: 47023
Max items filter: 450
Applying score... 450
Sorting by category... 450
Creating rank column... 450
Applying final columns... 450
Creating collection file...
Collection created.

######### Generating 10 12 06 Spike Day Guide - Up to R$50 #########
Repeated items across mic